# Human Segmentation using UNET

In [51]:
import os
import cv2
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#### Seeding

In [52]:
np.random.seed(24)
tf.random.set_seed(24)

#### Hyperparameters

In [80]:
batch_size = 8
learning_rate = 1e-4
epochs = 7

model_path = os.path.join("models", "unet-model.keras")
csv_log_path = os.path.join("models", "log.csv")

#### Loading the dataset

In [58]:
def load_dataset(path, split = 0.2):
    images = sorted(glob(os.path.join(path, "images/", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks/", "*.png")))
    
    split_size = int(len(images) * split)
    
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=24)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=24)
    
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=24)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=24)
    
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

dataset_path = ''
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
print(f'Train: {len(train_x)} - {len(train_y)}')
print(f'Valid: {len(valid_x)} - {len(valid_y)}')
print(f'Test: {len(test_x)} - {len(test_y)}')

Train: 1601 - 1601
Valid: 533 - 533
Test: 533 - 533


### Normalize data and create a batch sized dataset

In [59]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [60]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    
    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([256, 256, 3])
    y.set_shape([256, 256, 1])
    
    return x, y

def tf_dataset(x, y, batch = 8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(5)
    return dataset

In [61]:
train_dataset = tf_dataset(train_x, train_y, batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

#### Building the Model

In [76]:
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger

In [66]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2,2))(x)
    return x, p

In [67]:
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 3, strides = 2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [72]:
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    
    b1 = conv_block(p4, 1024)
    
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    
    model = Model(inputs, outputs, name="UNET")
    return model

#### Compiling the model

In [73]:
input_shape = (256, 256, 3)
model = build_unet(input_shape)
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])

In [74]:
model.summary()

Model: "UNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_76 (Conv2D)  │ (None, 256, 256,  │      1,792 │ input_layer_4[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_76[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_72       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_77 (Conv2D)  │ (None, 256, 256,  │     36,928 │ activation_72[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_77[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_73       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_16    │ (None, 128, 128,  │          0 │ activation_73[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_78 (Conv2D)  │ (None, 128, 128,  │     73,856 │ max_pooling2d_16… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_78[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_74       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_79 (Conv2D)  │ (None, 128, 128,  │    147,584 │ activation_74[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_79[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_75       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_17    │ (None, 64, 64,    │          0 │ activation_75[0]… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_80 (Conv2D)  │ (None, 64, 64,    │    295,168 │ max_pooling2d_17… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_80[0][0] 

 Total params: 34,536,897 (131.75 MB)

 Trainable params: 34,525,121 (131.70 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [81]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    CSVLogger(csv_log_path)
]

model.fit(
    train_dataset,
    epochs = epochs,
    validation_data = valid_dataset,
    callbacks = callbacks
)

Epoch 1/7
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.8508 - loss: 0.3442 
Epoch 1: val_loss improved from inf to 0.81827, saving model to models\unet-model.keras
201/201 ━━━━━━━━━━━━━━━━━━━━ 4151s 21s/step - accuracy: 0.8508 - loss: 0.3441 - val_accuracy: 0.7511 - val_loss: 0.8183
Epoch 2/7
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.8661 - loss: 0.3081 
Epoch 2: val_loss improved from 0.81827 to 0.63985, saving model to models\unet-model.keras
201/201 ━━━━━━━━━━━━━━━━━━━━ 3310s 16s/step - accuracy: 0.8661 - loss: 0.3081 - val_accuracy: 0.8014 - val_loss: 0.6398
Epoch 3/7
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.8823 - loss: 0.2754 
Epoch 3: val_loss improved from 0.63985 to 0.30121, saving model to models\unet-model.keras
201/201 ━━━━━━━━━━━━━━━━━━━━ 3186s 16s/step - accuracy: 0.8824 - loss: 0.2753 - val_accuracy: 0.8801 - val_loss: 0.3012
Epoch 4/7
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.8945 - loss: 0.2481 
Epoch 4: val_loss did no

In [82]:
test_dataset = tf_dataset(test_x, test_y, batch_size)

In [83]:
model.evaluate(test_dataset)

67/67 ━━━━━━━━━━━━━━━━━━━━ 235s 3s/step - accuracy: 0.8825 - loss: 0.2953


[0.2966804802417755, 0.8836576342582703]

In [99]:
def save_results(image, predicted_image, save_image_path):
    predicted_image = np.expand_dims(predicted_image, axis=-1)
    predicted_image = np.concatenate([predicted_image, predicted_image, predicted_image], axis=-1)
    predicted_image = predicted_image * 255
    
    line = np.ones((256, 10, 3)) * 255
    
    cat_images = np.concatenate([image, line, predicted_image], axis=1)
    cv2.imwrite(save_image_path, cat_images)
    
    
i = 0
for x, y in zip(test_x, test_y):
    if i == 15: break
    
    name = x.split("images\\")[-1]

    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (256, 256))
    x = image/255.0
    x = np.expand_dims(x, axis=0)
    
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (256, 256))
    
    y_pred = model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)
    
    save_predicted_path = os.path.join("results", name)
    save_results(image, y_pred, save_predicted_path)
    
    i += 1